# Intrusion Detection System using Autoencoders (Semi-Supervised Learning)

In [3]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.7/252.7 MB 8.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 8.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 8.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.4/670.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 8.7 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[

In [1]:
# Step 1: Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix


In [2]:
# Step 2: Load the dataset
df = pd.read_csv("UNSW-NB15P-MM-SAMPLE.csv")  # Update path as needed

In [3]:
# Step 3: Split into Normal (Dn) and Attack (Da)
Dn = df[df['Class'] == 0].drop(columns=['Class'])
Da = df[df['Class'] == 1].drop(columns=['Class'])

In [4]:
# Step 4: Split Dn into 80% training and 20% test
Dntr, Dnts = train_test_split(Dn, test_size=0.2, random_state=42)

In [5]:
# Step 5: Combine Dnts and Da to form test set (Dts)
Dts = pd.concat([Dnts, Da], ignore_index=True)
Dts_labels = [0]*len(Dnts) + [1]*len(Da)

In [6]:
# Step 6: Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(Dntr)
X_test = scaler.transform(Dts)

In [7]:
# Step 7: Define Autoencoder
input_dim = X_train.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(32, activation="relu")(input_layer)
encoded = Dense(16, activation="relu")(encoded)
decoded = Dense(32, activation="relu")(encoded)
output_layer = Dense(input_dim, activation="linear")(decoded)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(0.001), loss='mse')

In [8]:
# Step 8: Train the autoencoder on normal traffic
autoencoder.fit(X_train, X_train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_split=0.1,
                verbose=1)

Epoch 1/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 705us/step - loss: 0.5146 - val_loss: 0.1363
Epoch 2/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 0s 457us/step - loss: 0.1162 - val_loss: 0.1059
Epoch 3/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 0s 478us/step - loss: 0.0875 - val_loss: 0.0750
Epoch 4/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/step - loss: 0.0757 - val_loss: 0.0648
Epoch 5/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 507us/step - loss: 0.0627 - val_loss: 0.0581
Epoch 6/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - loss: 0.0571 - val_loss: 0.0516
Epoch 7/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 514us/step - loss: 0.0475 - val_loss: 0.0468
Epoch 8/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - loss: 0.0437 - val_loss: 0.0515
Epoch 9/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 546us/step - loss: 0.0402 - val_loss: 0.0390
Epoch 10/10
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 560us/step - loss: 0.0349 - val_loss: 0.0347


In [9]:
# Step 9: Predict on both training and combined test data
X_train_pred = autoencoder.predict(X_train)
X_test_pred = autoencoder.predict(X_test)

8570/8570 ━━━━━━━━━━━━━━━━━━━━ 2s 181us/step
2837/2837 ━━━━━━━━━━━━━━━━━━━━ 0s 161us/step


In [10]:
# Step 10: Compute Reconstruction Errors
train_errors = np.mean(np.square(X_train - X_train_pred), axis=1)
test_errors = np.mean(np.square(X_test - X_test_pred), axis=1)

In [11]:
# Step 11: Thresholding
threshold = np.percentile(train_errors, 95)
predicted_labels = (test_errors > threshold).astype(int)

In [12]:
# Step 12: Evaluation
print("Train Mean Error:", np.mean(train_errors))
print("Test Mean Error:", np.mean(test_errors))
print("\nConfusion Matrix:\n", confusion_matrix(Dts_labels, predicted_labels))
print("\nClassification Report:\n", classification_report(Dts_labels, predicted_labels))
print("\nROC-AUC Score:", roc_auc_score(Dts_labels, test_errors))

Train Mean Error: 0.03300393458801275
Test Mean Error: 0.35155288240900207

Confusion Matrix:
 [[65062  3495]
 [  799 21416]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.95      0.97     68557
           1       0.86      0.96      0.91     22215

    accuracy                           0.95     90772
   macro avg       0.92      0.96      0.94     90772
weighted avg       0.96      0.95      0.95     90772


ROC-AUC Score: 0.9892369289459102
